### Module Importation and DataFrame Setup

In [1]:
# Reset the model for re-run
from tensorflow.keras import backend as K
K.clear_session()

# Import dependencies
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
import keras_tuner as kt
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Read the CSV file from the Dataset folder into a Pandas DataFrame
dfSurvivals = pd.read_csv(
    Path("../CSV_OUTPUT/Base_Cleaned_DS_CSV.csv")
)

In [3]:
# Review the DataFrame
dfSurvivals.head()

,Passenger Id,Title,First Name,Last Name,Sex,Age,Sibling/Spouse Aboard,Parent/Children Aboard,Passenger Class,Fare,Embarkation Port,Survival Boat,Body Number,Survived,age_group,family_size
0,1,Miss,Elisabeth Walton,Allen,female,29.0000,0,0,1,211.3375,S,2,S,1,Adults,0
1,2,Master,Hudson Trevor,Allison,male,0.9167,1,2,1,151.5500,S,11,S,1,Children,3
2,3,Miss,Helen Loraine,Allison,female,2.0000,1,2,1,151.5500,S,DNS,BNR,0,Children,3
3,4,Mr,Hudson Joshua Creighton,Allison,male,30.0000,1,2,1,151.5500,S,DNS,135,0,Adults,3
4,5,Mrs,Hudson J C (Bessie Waldo Daniels),Allison,female,25.0000,1,2,1,151.5500,S,DNS,BNR,0,Adults,3


In [4]:
# Print DataFrame information to ensure no null values and check datatypes
dfSurvivals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308 entries, 0 to 1307
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Passenger Id            1308 non-null   int64  
 1   Title                   1308 non-null   object 
 2   First Name              1308 non-null   object 
 3   Last Name               1308 non-null   object 
 4   Sex                     1308 non-null   object 
 5   Age                     1308 non-null   float64
 6   Sibling/Spouse Aboard   1308 non-null   int64  
 7   Parent/Children Aboard  1308 non-null   int64  
 8   Passenger Class         1308 non-null   int64  
 9   Fare                    1308 non-null   float64
 10  Embarkation Port        1308 non-null   object 
 11  Survival Boat           1308 non-null   object 
 12  Body Number             1308 non-null   object 
 13  Survived                1308 non-null   int64  
 14  age_group               1308 non-null   

In [5]:
# Preprocessing for features which are currently objects
labelencoder_X = LabelEncoder()
dfSurvivals["Title"] = labelencoder_X.fit_transform(dfSurvivals["Title"])
dfSurvivals["Last Name"] = labelencoder_X.fit_transform(dfSurvivals["Last Name"])
dfSurvivals["Sex"] = labelencoder_X.fit_transform(dfSurvivals["Sex"])
dfSurvivals["Embarkation Port"] = labelencoder_X.fit_transform(dfSurvivals["Embarkation Port"])
dfSurvivals["age_group"] = labelencoder_X.fit_transform(dfSurvivals["age_group"])

In [6]:
# Drop columns
dfSurvivals = dfSurvivals.drop(['Passenger Id', 'First Name', 'Survival Boat', 'Body Number'], axis=1)
dfSurvivals.head()

,Title,Last Name,Sex,Age,Sibling/Spouse Aboard,Parent/Children Aboard,Passenger Class,Fare,Embarkation Port,Survived,age_group,family_size
0,9,15,0,29.0000,0,0,1,211.3375,2,1,0,0
1,8,16,1,0.9167,1,2,1,151.5500,2,1,1,3
2,9,16,0,2.0000,1,2,1,151.5500,2,0,1,3
3,12,16,1,30.0000,1,2,1,151.5500,2,0,0,3
4,13,16,0,25.0000,1,2,1,151.5500,2,0,0,3


In [7]:
# Ensure all columns are numerical
dfSurvivals.dtypes

Title                       int32
Last Name                   int32
Sex                         int32
Age                       float64
Sibling/Spouse Aboard       int64
Parent/Children Aboard      int64
Passenger Class             int64
Fare                      float64
Embarkation Port            int32
Survived                    int64
age_group                   int32
family_size                 int64
dtype: object

### Oversample to Correct Imbalance in Dataset
#### Oversampling code example used from https://www.section.io/engineering-education/how-to-handle-imbalanced-data-in-deep-neural-networks/#building-the-deep-neural-network-model-using-an-imbalanced-dataset

In [8]:
# Count the number of data samples in each class
count_majority_class, count_minority_class = dfSurvivals.Survived.value_counts()

In [9]:
# Save majority and minority classes in new variables
# In Survived column 0=No and 1=Yes 

df_majority_class = dfSurvivals[dfSurvivals['Survived'] == 0]
df_minority_class = dfSurvivals[dfSurvivals['Survived'] == 1]

In [10]:
# Oversample the minority data
df_class_oversample = df_minority_class.sample(count_majority_class, replace=True)

In [11]:
# Concatenate the oversampled minority class with the majority class
df_balanced_os = pd.concat([df_class_oversample, df_majority_class], axis=0)

In [12]:
# Count number of data samples in new balanced dataset
print('Number of data samples after over-sampling:')
print(df_balanced_os.Survived.value_counts())

Number of data samples after over-sampling:
1    808
0    808
Name: Survived, dtype: int64


In [13]:
# Review the DataFrame
df_balanced_os.head()

,Title,Last Name,Sex,Age,Sibling/Spouse Aboard,Parent/Children Aboard,Passenger Class,Fare,Embarkation Port,Survived,age_group,family_size
503,12,517,1,19.0,0,0,2,10.5000,2,1,0,0
395,9,213,0,18.0,0,1,2,23.0000,2,1,3,1
738,9,179,0,17.0,0,1,3,16.1000,2,1,3,1
120,12,261,1,43.0,1,0,1,27.7208,0,1,0,1
1119,12,631,1,25.0,1,0,3,7.7750,2,1,0,1


### Split the data into X and y and then into testing and training sets

In [14]:
# Split the data into X (features) and y (target)

# Set the y variable, which is the target
y = df_balanced_os['Survived']

# Set the X variable, which includes all features escept the target
X = df_balanced_os.drop(columns=['Survived'])

In [15]:
# Split into testing and training sets using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train, and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_dim = input_dim = X_train.shape[1]

def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=input_dim))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 2)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
   
    # Add final layer with sigmoid activation function
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy","Precision","Recall"])

    return nn_model

In [18]:
# Set up the kerastuner then allow it to search for best hyperparameters
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True)

# Initialize EarlyStopping callback to monitor the validation loss
# Training will stop if the validation loss doesn't improve for 3 consecutive epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Perform hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

Trial 60 Complete [00h 00m 04s]
val_accuracy: 0.7896039485931396

Best val_accuracy So Far: 0.8143564462661743
Total elapsed time: 00h 02m 30s
INFO:tensorflow:Oracle triggered exit


In [19]:
# Get the hyperparameters of the best model
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 9,
 'num_layers': 2,
 'units_0': 7,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'units_1': 1,
 'tuner/trial_id': '0015'}

In [20]:
# Get the best model
best_model = tuner.get_best_models(1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 108       
                                                                 
 dense_1 (Dense)             (None, 7)                 70        
                                                                 
 dense_2 (Dense)             (None, 1)                 8         
                                                                 
 dense_3 (Dense)             (None, 1)                 2         
                                                                 
Total params: 188 (752.00 Byte)
Trainable params: 188 (752.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# Train the model
fit_model = best_model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
38/38 [==============================] - 1s 1ms/step - loss: 0.4981 - accuracy: 0.7904 - precision: 0.8084 - recall: 0.7603
Epoch 2/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4961 - accuracy: 0.7904 - precision: 0.8106 - recall: 0.7570
Epoch 3/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4941 - accuracy: 0.7896 - precision: 0.8092 - recall: 0.7570
Epoch 4/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4932 - accuracy: 0.7880 - precision: 0.8021 - recall: 0.7636
Epoch 5/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4905 - accuracy: 0.7921 - precision: 0.8070 - recall: 0.7669
Epoch 6/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4889 - accuracy: 0.7896 - precision: 0.8114 - recall: 0.7537
Epoch 7/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4869 - accuracy: 0.7913 - precision: 0.8088 - recall: 0.7620
Epoch 8/100
38/38 [=============================

38/38 [==============================] - 0s 1ms/step - loss: 0.4384 - accuracy: 0.8119 - precision: 0.8421 - recall: 0.7669
Epoch 61/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4371 - accuracy: 0.8094 - precision: 0.8450 - recall: 0.7570
Epoch 62/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4362 - accuracy: 0.8144 - precision: 0.8519 - recall: 0.7603
Epoch 63/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4363 - accuracy: 0.8135 - precision: 0.8477 - recall: 0.7636
Epoch 64/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4358 - accuracy: 0.8119 - precision: 0.8484 - recall: 0.7587
Epoch 65/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4363 - accuracy: 0.8144 - precision: 0.8493 - recall: 0.7636
Epoch 66/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4361 - accuracy: 0.8102 - precision: 0.8531 - recall: 0.7488
Epoch 67/100
38/38 [==============================] - 

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_precision, model_recall = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, Precision: {model_precision}, Recall: {model_recall}")

13/13 - 0s - loss: 0.4306 - accuracy: 0.8218 - precision: 0.8619 - recall: 0.7685 - 241ms/epoch - 19ms/step
Loss: 0.43059444427490234, Accuracy: 0.8217821717262268, Precision: 0.8618784546852112, Recall: 0.7684729099273682


In [23]:
# Use the model to make predictions on test data
y_pred_prob = best_model.predict(X_test_scaled).flatten()
y_pred = [1 if prob >= 0.5 else 0 for prob in y_pred_prob]

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Generate a classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

13/13 [==============================] - 0s 917us/step
Confusion Matrix:
[[176  25]
 [ 47 156]]
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.88      0.83       201
           1       0.86      0.77      0.81       203

    accuracy                           0.82       404
   macro avg       0.83      0.82      0.82       404
weighted avg       0.83      0.82      0.82       404



In [24]:
# Export our model to keras file
best_model.save('OUTPUT/Titanic_survival_mod2_reprocessed_opt1.keras')